<a href="https://colab.research.google.com/github/rcnewman/cv_gesture_recognition_posenet/blob/master/PoseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PoseNet-python Notebook Source: https://colab.research.google.com/drive/1Ha7-lt-WwcCx961cuatjOzQ-7GLY91aQ#scrollTo=k_qlzhSXN3D7
# install
!git clone https://github.com/rcnewman/posenet-python.git #fork from https://www.github.com/rwightman/posenet-python 
%cd posenet-python
!pip3 install tensorflow-gpu==1.15
!pip3 install scipy
!pip3 install pyyaml
!pip3 install opencv-python
!pip3 install ipykernel
!pip3 install msgpack-numpy

fatal: destination path 'posenet-python' already exists and is not an empty directory.
/content/posenet-python


In [2]:
#CONFIG
max_pose_detections = 3
min_pose_score = 0.25
min_part_score = 0.25
SAVE_KEYPOINTS = True
DEBUG = False
SAVE_VIDEO = True # Have not gotten output saving working on my end yet TODO: Output saving

In [4]:
# PoseNet python sample program
import tensorflow as tf
import cv2
import time
import argparse
import os
import posenet
import glob
import msgpack
import msgpack_numpy as m
m.patch() #for handling numpy arrays w/ msgpack
print('INIT:')
from google.colab import drive
drive.mount('/content/drive')
drivepath="/content/drive/My Drive/POSENET/"
driveinfile = drivepath + 'input.mp4'
video_path = glob.glob(drivepath + 'vids/*.mp4')
save_video_path = drivepath + 'output_vids/'
keypts_path = drivepath +'keypts/'
# VideoReaderWriter
fourcc = cv2.VideoWriter_fourcc('h','2','6', '4')


model = 101
###scale_factor = 1.0
scale_factor = 0.4

with tf.Session() as sess:
    print('MODEL-INIT:')
    ####model_cfg, model_outputs = posenet.load_model(args.model, sess)
    model_cfg, model_outputs = posenet.load_model(model, sess)
    output_stride = model_cfg['output_stride']
    start = time.time()
    print('START:')
    for vid in video_path:
      cap = cv2.VideoCapture(vid)
      incnt = 0
      keypoints_save = []
      if SAVE_VIDEO:
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        driveoutfile = save_video_path + os.path.basename(vid)
        video = cv2.VideoWriter(driveoutfile, fourcc, fps, (width, height))
      while True:
          incnt = incnt + 1
          try: input_image, draw_image, output_scale = posenet.read_cap(
                  cap, scale_factor=scale_factor, output_stride=output_stride)
          except:break
          heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
              model_outputs,
              feed_dict={'image:0': input_image}
          )
          pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
              heatmaps_result.squeeze(axis=0),
              offsets_result.squeeze(axis=0),
              displacement_fwd_result.squeeze(axis=0),
              displacement_bwd_result.squeeze(axis=0),
              output_stride=output_stride,
              max_pose_detections=max_pose_detections,
              min_pose_score=min_pose_score)

          keypoint_coords *= output_scale

          if SAVE_KEYPOINTS:
            keypoints_save.append(keypoint_coords)
          
          if SAVE_VIDEO:
            draw_image = posenet.draw_skel_and_kp(
                    draw_image, pose_scores, keypoint_scores, keypoint_coords,
                    min_pose_score=min_pose_score, min_part_score=min_part_score)

            video.write(draw_image)

          if incnt % 100 == 0:        
              print("cnt=", incnt, "fps=", incnt / (time.time() - start) )

          if DEBUG:
              #debug
              for pi in range(len(pose_scores)):
                  if pose_scores[pi] == 0.:
                      break
                  print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                  for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                      print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
      
      cap.release()

      if SAVE_KEYPOINTS:
        with open(keypts_path + os.path.basename(vid) + "_pts.msgpack", 'wb') as f:
          packed = msgpack.packb(keypoints_save)
          f.write(packed)

      if SAVE_VIDEO:
         video.release()

print('END:')


INIT:
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MODEL-INIT:


START:
cnt= 100 fps= 5.488606921495356
cnt= 200 fps= 5.484048791840086
END:


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense

# TODO: Actually set-up the model beyond skeleton


#num_gestures = 11 #10 gestures + null
#batch = 32
#num_epochs = 3


# model = Sequential()
# model.add(Input()
# model.add(LSTM())
# model.add(LSTM())
# model.add(Dense(num_gestures, activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# X_train, X_valid, Y_train, Y_valid = subject_wise_split(X,Y)

# model.fit(X_train, Y_train, batch_size=batch, epochs=num_epochs,verbose=1)

# score, acc = model.evaluate(X_valid,Y_valid)